In [1]:
import random
def initialize_deck():
    command_zone = ["Thalia and the Gitrog Monster"]
    field = []
    field.append('The Gitrog Monster')
    field.append('Putrid Imp')
    hand = ['Dakmor Salvage']
    grave = []
    exile = []


    deck = []
    game_zones = [deck,grave,hand,field,exile]
    for i in range(63):
        game_zones[random.randint(0,4)].append('x')


    for i in range(31):
        game_zones[random.randint(0,4)].append('land')
    kzone =random.randint(0,2)
    if kzone == 1:
        kzone = 2
    game_zones[kzone].append('Kozilek')


    random.shuffle(deck)
    draw_triggers = 0
    shuffle_triggers = 0

    game_state = [deck, grave, hand, draw_triggers, shuffle_triggers, field, exile, command_zone]
    return game_state

In [2]:
def dredge(zones):  
    trigger = False
    if len(zones[0]) ==0:
        print('empty')
    else:
        if zones[0][0] == 'land':
            trigger = True
        zones[1].append(zones[0][0])
        zones[0].remove(zones[0][0])
        
    if len(zones[0]) ==0:
        print('empty')
    else:
        if zones[0][0] == 'land':
            trigger = True
        zones[1].append(zones[0][0])
        zones[0].remove(zones[0][0])
        
    if trigger:
        zones[3] += 1
#         gitrog_trigger(zones[1][0], zones)
    return zones
    
# def gitrog_trigger(card, zones):
#     if card == 'land' or card == 'Dakmor Salvage':
#         zones[3] += 1
#     return zones

def resolve_draw_trigger(zones,init):
    zones[3] -=1
    if zones[3] < 0:
        print_state(init)
        print("Deck Out!!!")
        raise Exception('Deck out!')
    zones[2].append(zones[0][0])
    zones[0].remove(zones[0][0])
    return zones
           
def resolve_kozilek_trigger(zones):
    zones[4] -= 1
    if zones[4] < 0:
        print(zones)
        raise Exception('Illegal Shuffle!')
    for i in zones[1]:
        zones[0].append(i)
    zones[1] = []
    random.shuffle(zones[0])
    return zones

def discard_lands(zones):
    for card in zones[2]:
        if correct_lands(zones[0],zones[1]) <= 3:
            if card == 'land':
                zones[1].append(card)
                zones[2].remove(card)
                zones[3] += 1
    return zones
            
            
    
def correct_lands(deck, grave):
    num_lands = 0
    for i in deck:
        if i=='land':
            num_lands += 1
    for j in grave:
        if j =='land':
            num_lands += 1
    return num_lands

def count_lands_circulation(deck, grave, hand):
    num_lands_d = 0
    num_lands_g = 0
    num_lands_h = 0
    for i in deck:
        if i=='land':
            num_lands_d += 1
    for i in grave:
        if i=='land':
            num_lands_g += 1
    for i in hand:
        if i=='land':
            num_lands_h += 1
#     print(num_lands_d, num_lands_g, num_lands_h)
    num_lands = num_lands_d + num_lands_g + num_lands_h
    return num_lands, num_lands_d,num_lands_g, num_lands_h

def admit_fail_states(game_state):
    fails = False
    reason = ""
    lds = count_lands_circulation(game_state[0],game_state[1],game_state[2])
    if lds[0] < 3:
#         print(lds)
        fails = True
        reason = "Too Few Lands in Circulation"
#     if 'Kozilek' not in game_state[0] or 'Kozilek' not in game_state[1] or 'Kozilek' not in game_state[2]:
#         fails = True
#         reason = "Kozilek on field or in exile"
#     print(game_state)
    
    return fails, reason

def print_state(game_state):
    print("Deck =",game_state[0],",")
    print("Grave =", game_state[1],",")
    print("Hand =", game_state[2],",")
    print("Draw_Triggers =",game_state[3],",")
    print("Shuffle_Triggers =", game_state[4],",")
    print("Field =",game_state[5],",")
    print("Exile =", game_state[6],",")
    print("Command_Zone =", game_state[7])
    print("Cards in deck =",len(game_state[0]))

    
    return

In [3]:
# assume gitrog and putrid imp on field, and dakmor salvage in hand

# while draw_triggers < len(game_state[0]):
def gitrog_loop(game_state):
    init_state = game_state
    edge_cases = admit_fail_states(game_state)
    if edge_cases[0]:
        raise Exception(edge_cases[1])

    while game_state[3] < len(game_state[0]):
    # set initial condition of 3 lands between grave and deck
        discard_lands(game_state)

#     dredge until kozilek in grave or hand or if kozilek is the last card in deck
        while len(game_state[0]) > 1:
            halt = False
            for i in game_state[1]:
                if i == 'Kozilek':
                    halt = True
            for j in game_state[2]:
                if j == 'Kozilek':
                    halt = True
#             if 'Kozilek' not in game_state[1] or 'Kozilek' not in game_state[2]:
            if not halt:
#                 print(game_state[2])
                dredge(game_state)
            else:
                break
#     case if kozilek is last card in deck        
        if len(game_state[0])==1:
            if game_state[0] == 'Kozilek':
#             print("Kozilek is last card in deck")
                resolve_draw_trigger(game_state, init_state)
#         discard kozilek
        for i in game_state[2]:
            if i == 'Kozilek':
                game_state[1].append(i)
                game_state[2].remove(i)
        game_state[4] += 1
        
#         onto resolving shuffle part of loop
        while len(game_state[0]) >= 3:
            draws = game_state[3]
            dredge(game_state)
            if game_state[3] > draws:
                resolve_draw_trigger(game_state, init_state)
        resolve_kozilek_trigger(game_state)  
#         print(game_state)
    return game_state, init_state
        

In [4]:
# 76 million to test all permutations
epochs = 62000000
for i in range(epochs):
    g_state = initialize_deck()
    
#     if (len(initialize_deck()[0])) ==1:
#         print("too few cards")
#     if len(g_state[0]) < 3:
#         print(len(g_state[0]))
    
#     print("Iteration", i)
#     print(g_state)
#     print("end state")
    res = gitrog_loop(g_state)
    if i % (epochs/ 1000) == 0:
        print(i / (epochs / 100), " percent tested")
        
#     res = gitrog_loop(res)

#     print_state(g_state)


0.0  percent tested
0.1  percent tested
0.2  percent tested
0.3  percent tested
0.4  percent tested
0.5  percent tested
0.6  percent tested
0.7  percent tested
0.8  percent tested
0.9  percent tested
1.0  percent tested
1.1  percent tested
1.2  percent tested
1.3  percent tested
1.4  percent tested
1.5  percent tested
1.6  percent tested
1.7  percent tested
1.8  percent tested
1.9  percent tested
2.0  percent tested
2.1  percent tested
2.2  percent tested
2.3  percent tested
2.4  percent tested
2.5  percent tested
2.6  percent tested
2.7  percent tested
2.8  percent tested
2.9  percent tested
3.0  percent tested
3.1  percent tested
3.2  percent tested
3.3  percent tested
3.4  percent tested
3.5  percent tested
3.6  percent tested
3.7  percent tested
3.8  percent tested
3.9  percent tested
4.0  percent tested
4.1  percent tested
4.2  percent tested
4.3  percent tested
4.4  percent tested
4.5  percent tested
4.6  percent tested
4.7  percent tested
4.8  percent tested
4.9  percent tested


39.5  percent tested
39.6  percent tested
39.7  percent tested
39.8  percent tested
39.9  percent tested
40.0  percent tested
40.1  percent tested
40.2  percent tested
40.3  percent tested
40.4  percent tested
40.5  percent tested
40.6  percent tested
40.7  percent tested
40.8  percent tested
40.9  percent tested
41.0  percent tested
41.1  percent tested
41.2  percent tested
41.3  percent tested
41.4  percent tested


KeyboardInterrupt: 

In [ ]:
# Deck = ['land'] ,
# Grave = ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x'] ,
# Hand = ['Dakmor Salvage', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'Kozilek'] ,
# Draw_Triggers = -1 ,
# Shuffle_Triggers = 0 ,
# Field = ['The Gitrog Monster', 'Putrid Imp', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land'] ,
# Exile = ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land'] ,
# Command_Zone = ['Thalia and the Gitrog Monster']
# g_state = [Deck, Grave, Hand, Draw_Triggers, Shuffle_Triggers, Field, Exile, Command_Zone]
g_state = [['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'Kozilek', 'x'], 
           ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land'], 
           ['Dakmor Salvage', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land'], 
           0, 0, 
           ['The Gitrog Monster', 'Putrid Imp', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land'], 
           ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land'], 
           ['Thalia and the Gitrog Monster']]
# print_state(g_state)
gitrog_loop(g_state)
# print(g_state)

In [ ]:
Deck = ['x'] ,
Grave = ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'Kozilek'] ,
Hand = ['Dakmor Salvage', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land'] ,
Draw_Triggers = 0 ,
Shuffle_Triggers = 0 ,
Field = ['The Gitrog Monster', 'Putrid Imp', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land'] ,
Exile = ['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land', 'land'] ,
Command_Zone = ['Thalia and the Gitrog Monster']
g_state = [Deck, Grave, Hand, Draw_Triggers, Shuffle_Triggers, Field, Exile, Command_Zone]
gitrog_loop(g_state)